## Start

In [5]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:  

[[total_active_users_per_day_full_table]]

[[entries_st_mobile]]

[[citizens_dir_mobile]]


### Table

In [6]:
query_text = """--sql
    SELECT
        *
    FROM
        (SELECT
            total_act.report_date AS report_date,
            total_act.partner_uuid AS partner_uuid,
            total_act.citizen_id AS citizen_id,
            total_act.monetization AS monetization,
            total_act.subscriptions_state AS subscriptions_state,
            entr_st_m.ble_available AS ble_available,
            cit_dir_m.activated_at AS activated_at,
            total_act.city AS city,
            ROW_NUMBER() OVER (
                    PARTITION BY
                        toStartOfMonth(total_act.report_date),
                        total_act.citizen_id
                    ORDER BY total_act.report_date
                ) = 1 AS is_first_mention_in_month
        FROM db1.total_active_users_per_day_full_table AS total_act
        LEFT JOIN db1.`entries_st_mobile_ch` AS entr_st_m
            ON `entr_st_m`.`report_date` = total_act.`report_date`
            AND`entr_st_m`.`address_uuid` = total_act.`address_uuid`
        LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                    ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
        WHERE report_date = '2025-08-13'
            )
    WHERE is_first_mention_in_month = 1
    limit 10
"""

ch.get_schema(query_text)

(
    `report_date` Date,
    `partner_uuid` String,
    `citizen_id` Int32,
    `monetization` UInt8,
    `subscriptions_state` String,
    `ble_available` String,
    `activated_at` String,
    `city` String,
    `is_first_mention_in_month` UInt8
)


In [ ]:
query_text = """--sql
CREATE TABLE db1.total_active_users_first_mention
(
    `report_date` Date,
    `partner_uuid` String,
    `citizen_id` Int32,
    `monetization` UInt8,
    `subscriptions_state` String,
    `ble_available` String,
    `activated_at` String,
    `city` String,
    `is_first_mention_in_month` UInt8
)
ENGINE = MergeTree()
ORDER BY partner_uuid
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [10]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.total_active_users_first_mention_mv
    REFRESH EVERY 1 DAY OFFSET 4 HOUR 20 MINUTE TO db1.total_active_users_first_mention AS
          SELECT
        *
    FROM
        (SELECT
            total_act.report_date AS report_date,
            total_act.partner_uuid AS partner_uuid,
            total_act.citizen_id AS citizen_id,
            total_act.monetization AS monetization,
            total_act.subscriptions_state AS subscriptions_state,
            entr_st_m.ble_available AS ble_available,
            cit_dir_m.activated_at AS activated_at,
            total_act.city AS city,
            ROW_NUMBER() OVER (
                    PARTITION BY
                        toStartOfMonth(total_act.report_date),
                        total_act.citizen_id
                    ORDER BY total_act.report_date
                ) = 1 AS is_first_mention_in_month
        FROM db1.total_active_users_per_day_full_table AS total_act
        LEFT JOIN db1.`entries_st_mobile_ch` AS entr_st_m
            ON `entr_st_m`.`report_date` = total_act.`report_date`
            AND`entr_st_m`.`address_uuid` = total_act.`address_uuid`
        LEFT JOIN db1.`citizens_dir_mobile_ch` AS cit_dir_m
                    ON `cit_dir_m`.`citizen_id` = total_act.`citizen_id`
            )
    WHERE is_first_mention_in_month = 1
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [11]:
query_text = """--sql
    SELECT
        *
    FROM db1.total_active_users_first_mention
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)

report_date,partner_uuid,citizen_id,monetization,subscriptions_state,ble_available,activated_at,city,is_first_mention_in_month
date,str,i64,i64,str,str,str,str,i64
2025-08-14,"""""",122864,1,"""""","""false""","""""","""Новокузнецк""",1
2025-08-14,"""""",205839,1,"""""","""false""","""2021-10-07 08:22:12""","""Новокузнецк""",1
2025-08-14,"""""",200091,1,"""""","""false""","""2021-04-10 14:19:06""","""Волжский""",1
2025-08-14,"""""",179087,1,"""""","""false""","""2021-03-18 13:59:19""","""Уфа""",1
2025-08-14,"""""",82689,1,"""""","""false""","""2022-03-29 14:58:45""","""Нур-Султан (Астана)""",1
…,…,…,…,…,…,…,…,…
2025-08-14,"""""",2071742,0,"""""","""""","""""","""""",1
2025-08-14,"""""",2058415,0,"""""","""""","""""","""""",1
2025-08-14,"""""",988265,1,"""""","""false""","""2023-01-27 06:55:08""","""Новокузнецк""",1


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.total_active_users_per_day_full_table DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop mv

In [ ]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### drop ch

In [ ]:
query_text = """--sql
    DROP TABLE db1.total_active_users_per_day_full_table
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [ ]:
query_text = """
SYSTEM REFRESH VIEW db1.total_active_users_per_day_full_table_mv
"""

ch.query_run(query_text)